In [ ]:
####################################################
# Description: Pokemon battle Winner Predict Model #
# Author : KJ                                      #
# Created Date : 2018-11-21                        #
####################################################

In [27]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

from torch.utils.data import TensorDataset, DataLoader
from torch import optim

In [29]:
class Net(nn.Module):
    # Neual Network Model to predict which pokémon will win in Pokémon battle 
    def __init__(self):
        super(Net, self).__init__()
        
        def init_weights(m):
            if type(m) == nn.Linear:
                torch.nn.init.xavier_uniform(m.weight)
                m.bias.data.fill_(0.01)
                
        self.model = nn.Sequential(
            nn.Linear(18, 100),
            nn.ReLU(),
            nn.Linear(100, 1000),
            nn.ReLU(),
            nn.Linear(1000, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )
        
        self.model.apply(init_weights)
        
    def forward(self, x):
        
        return self.model(x)

In [35]:
def train(model, train_loader, optimizer, epoch):
    # training function
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        
        # TODO : find the best loss function
        criterion = nn.MSELoss(size_average=False) 
        loss = criterion(output, target)
        loss.backward()
        
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, test_loader):
    # testing function
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss +=  (output - target).pow(2).sum()
            pred = output[:, 0] < 0.5
            real = target[:, 0] < 0.5

            correct += (pred == real).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [36]:
TYPES = { np.nan : 0,
         'Normal' : 1, 
         'Fire' : 2, 
         'Water' : 3, 
         'Electric' : 4, 
         'Grass' : 5, 
         'Ice' : 6, 
         'Fighting' : 7, 
         'Poison' : 8, 
         'Ground' : 9, 
         'Flying' : 10, 
         'Psychic' : 11, 
         'Bug' : 12, 
         'Rock' : 13, 
         'Ghost' : 14, 
         'Dragon' : 15, 
         'Dark' : 16, 
         'Steel' : 17, 
         'Fairy' : 18 }

COLUMNS = [
        'Winner', #1
    
        'Legendary_x', #2
        'Legendary_y', #3
    
        'Type 1_x', #4
        'Type 2_x', #5
        'HP_x', #6
        'Attack_x', #7 
        'Defense_x', #8
        'Sp. Atk_x', #9
        'Sp. Def_x', #10
        'Speed_x', #11
    
        'Type 1_y', #12
        'Type 2_y', #13
        'HP_y', #14
        'Attack_y', #15
        'Defense_y', #16
        'Sp. Atk_y', #17
        'Sp. Def_y', #18
        'Speed_y' #19
]

In [37]:
class PokemonData():
    
    def __init__(self, combats_file_path='./data/combats.csv', pokemon_file_path='./data/pokemon.csv'):
        self.combat_file_path = combats_file_path
        self.pokemon_file_path = pokemon_file_path
        
    def get_prepared_data_set(self, train_path='', test_path=''):
        
        try:
            train_tensor = torch.load(path)
            self.train_loader = DataLoader(train_tensor, batch_size=8, shuffle=True, num_workers = 2)
        except Exception as e:
            raise Exception('Invaild train tensor path')
        
        print('Train data loading Complete!')
        
        try:
            test_tensor = torch.load(path)
            self.test_loader = DataLoader(test_tensor, batch_size=8, shuffle=False, num_workers = 2)
        except Exception as e:
            raise Exception('Invaild test tensor path')
            
        print('Test data loading Complete!')
        
    def read_file(self):
        try:
            combats = pd.read_csv(self.combat_file_path)
        except Exception as e:
            raise Exception('Invaild combat file path')
        
        print('Train data loading Complete!')
        
        try:
            pokemons = pd.read_csv(self.pokemon_file_path)
        except Exception as e:
            raise Exception('Invaild pokemon file path')
            
        print('Test data loading Complete!')
        
        return combats, pokemons

    def preprocessing(self, save=True):
        combats, pokemons = self.read_file()
            
        df = pd.merge(combats, pokemons, left_on='First_pokemon', right_on='#')
        df = pd.merge(df, pokemons, left_on='Second_pokemon', right_on='#')

        winner = df['Winner']
        df['Winner'] = np.where(winner == df['First_pokemon'], 1.0, 0.0)

        df.replace({   "Type 1_x": TYPES, 
                       "Type 2_x": TYPES,
                       "Type 1_y": TYPES,
                       "Type 2_y": TYPES }, inplace=True)

        normalization = {
            'Legendary_x' : 0, #3 
            'Legendary_y' : 0, #4
            
            'Type 1_x' : 0, #5
            'Type 2_x' : 0, #6
            'HP_x' : 0, #7
            'Attack_x' : 0, #8 
            'Defense_x' : 0, #9
            'Sp. Atk_x' : 0, #10
            'Sp. Def_x' : 0, #11
            'Speed_x' : 0, #12

            'Type 1_y' : 0, #13
            'Type 2_y' : 0, #14
            'HP_y' : 0, #15
            'Attack_y' : 0, #16
            'Defense_y' : 0, #17
            'Sp. Atk_y' : 0, #18
            'Sp. Def_y' : 0, #19
            'Speed_y' : 0 #20
        }
        
        # normalize
        for column in COLUMNS[3:]:
            normalization[column] = {
                'mean' : df[column].mean(),
                'max' : df[column].max(),
                'min' : df[column].min()
            }
            df[column] = (df[column] - df[column].mean()) / (df[column].max() - df[column].min())


        train_set = df.loc[:len(df) * 0.6 - 1, COLUMNS[1:]]
        train_label = df.loc[:len(df) * 0.6 - 1, COLUMNS[:1]]

        test_set = df.loc[len(df) * 0.6 : , COLUMNS[1:]]
        test_label =  df.loc[len(df) * 0.6 : , COLUMNS[:1]]

        train_set = torch.tensor(train_set.values.astype(np.float), dtype=torch.float,requires_grad=True)
        train_label = torch.tensor(train_label.values.astype(np.float), dtype=torch.float,requires_grad=True)
        train_tensor = TensorDataset(train_set, train_label)

        test_set = torch.tensor(test_set.values.astype(np.float), dtype=torch.float,requires_grad=False)
        test_label = torch.tensor(test_label.values.astype(np.float), dtype=torch.float,requires_grad=False)
        test_tensor = TensorDataset(test_set, test_label)
        
        if save == True:
            torch.save(train_tensor, 'train_set')
            torch.save(test_tensor, 'test_set')
            
        self.train_loader = DataLoader(train_tensor, batch_size=8, shuffle=True, num_workers = 2)
        self.test_loader = DataLoader(test_tensor, batch_size=8, shuffle=False, num_workers = 2)
    
    def postprocessing(self, save=True):
        pass
    

In [38]:
def main():
    pokemon_data = PokemonData()
    pokemon_data.preprocessing()
    # Or if you want to use prepared data
    # pokemon_data.get_prepared_data
    
    model = Net()
    
    # TODO : find appropriate learning rate and optimizer
    optimizer = optim.Adam(model.parameters(), lr=1.0e-5)

    epochs = 200

    for epoch in range(1, epochs + 1):
        train(model,  pokemon_data.train_loader, optimizer, epoch)
        test(model, pokemon_data.test_loader)

    torch.save(model.state_dict(), './model')

In [ ]:
if __name__ == '__main__':
    main()

Train data loading Complete!
Test data loading Complete!


C:\Users\iclab\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
C:\Users\iclab\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 1 [0/30000 (0%)]	Loss: 1.937883
Train Epoch: 1 [80/30000 (0%)]	Loss: 2.058570
Train Epoch: 1 [160/30000 (1%)]	Loss: 1.952085
Train Epoch: 1 [240/30000 (1%)]	Loss: 1.974180
Train Epoch: 1 [320/30000 (1%)]	Loss: 2.003916
Train Epoch: 1 [400/30000 (1%)]	Loss: 1.982729
Train Epoch: 1 [480/30000 (2%)]	Loss: 1.952928
Train Epoch: 1 [560/30000 (2%)]	Loss: 2.006582
Train Epoch: 1 [640/30000 (2%)]	Loss: 1.965901
Train Epoch: 1 [720/30000 (2%)]	Loss: 1.999836
Train Epoch: 1 [800/30000 (3%)]	Loss: 2.021289
Train Epoch: 1 [880/30000 (3%)]	Loss: 1.969964
Train Epoch: 1 [960/30000 (3%)]	Loss: 2.004156
Train Epoch: 1 [1040/30000 (3%)]	Loss: 1.997314
Train Epoch: 1 [1120/30000 (4%)]	Loss: 1.920820
Train Epoch: 1 [1200/30000 (4%)]	Loss: 1.950426
Train Epoch: 1 [1280/30000 (4%)]	Loss: 1.917307
Train Epoch: 1 [1360/30000 (5%)]	Loss: 1.958232
Train Epoch: 1 [1440/30000 (5%)]	Loss: 1.950170
Train Epoch: 1 [1520/30000 (5%)]	Loss: 1.991969
Train Epoch: 1 [1600/30000 (5%)]	Loss: 1.948159
Train Ep

Train Epoch: 1 [13440/30000 (45%)]	Loss: 1.059846
Train Epoch: 1 [13520/30000 (45%)]	Loss: 1.614435
Train Epoch: 1 [13600/30000 (45%)]	Loss: 1.400209
Train Epoch: 1 [13680/30000 (46%)]	Loss: 1.563286
Train Epoch: 1 [13760/30000 (46%)]	Loss: 1.778500
Train Epoch: 1 [13840/30000 (46%)]	Loss: 0.855031
Train Epoch: 1 [13920/30000 (46%)]	Loss: 1.171465
Train Epoch: 1 [14000/30000 (47%)]	Loss: 1.700731
Train Epoch: 1 [14080/30000 (47%)]	Loss: 1.203237
Train Epoch: 1 [14160/30000 (47%)]	Loss: 1.291298
Train Epoch: 1 [14240/30000 (47%)]	Loss: 1.689377
Train Epoch: 1 [14320/30000 (48%)]	Loss: 1.913697
Train Epoch: 1 [14400/30000 (48%)]	Loss: 2.188833
Train Epoch: 1 [14480/30000 (48%)]	Loss: 1.289765
Train Epoch: 1 [14560/30000 (49%)]	Loss: 1.619973
Train Epoch: 1 [14640/30000 (49%)]	Loss: 1.590678
Train Epoch: 1 [14720/30000 (49%)]	Loss: 1.676066
Train Epoch: 1 [14800/30000 (49%)]	Loss: 1.349273
Train Epoch: 1 [14880/30000 (50%)]	Loss: 1.505752
Train Epoch: 1 [14960/30000 (50%)]	Loss: 1.253050


Train Epoch: 1 [27040/30000 (90%)]	Loss: 0.825433
Train Epoch: 1 [27120/30000 (90%)]	Loss: 0.644021
Train Epoch: 1 [27200/30000 (91%)]	Loss: 0.336936
Train Epoch: 1 [27280/30000 (91%)]	Loss: 1.876200
Train Epoch: 1 [27360/30000 (91%)]	Loss: 0.999876
Train Epoch: 1 [27440/30000 (91%)]	Loss: 0.764473
Train Epoch: 1 [27520/30000 (92%)]	Loss: 2.028016
Train Epoch: 1 [27600/30000 (92%)]	Loss: 0.520628
Train Epoch: 1 [27680/30000 (92%)]	Loss: 1.136617
Train Epoch: 1 [27760/30000 (93%)]	Loss: 0.909760
Train Epoch: 1 [27840/30000 (93%)]	Loss: 0.941913
Train Epoch: 1 [27920/30000 (93%)]	Loss: 0.845079
Train Epoch: 1 [28000/30000 (93%)]	Loss: 1.222616
Train Epoch: 1 [28080/30000 (94%)]	Loss: 1.129463
Train Epoch: 1 [28160/30000 (94%)]	Loss: 0.508508
Train Epoch: 1 [28240/30000 (94%)]	Loss: 1.034513
Train Epoch: 1 [28320/30000 (94%)]	Loss: 0.673934
Train Epoch: 1 [28400/30000 (95%)]	Loss: 1.443377
Train Epoch: 1 [28480/30000 (95%)]	Loss: 1.200846
Train Epoch: 1 [28560/30000 (95%)]	Loss: 1.059489


...

Train Epoch: 75 [20960/30000 (70%)]	Loss: 0.001573
Train Epoch: 75 [21040/30000 (70%)]	Loss: 0.004331
Train Epoch: 75 [21120/30000 (70%)]	Loss: 0.009591
Train Epoch: 75 [21200/30000 (71%)]	Loss: 1.999999
Train Epoch: 75 [21280/30000 (71%)]	Loss: 1.000027
Train Epoch: 75 [21360/30000 (71%)]	Loss: 1.002767
Train Epoch: 75 [21440/30000 (71%)]	Loss: 0.157206
Train Epoch: 75 [21520/30000 (72%)]	Loss: 1.021067
Train Epoch: 75 [21600/30000 (72%)]	Loss: 0.000552
Train Epoch: 75 [21680/30000 (72%)]	Loss: 1.001758
Train Epoch: 75 [21760/30000 (73%)]	Loss: 0.006106
Train Epoch: 75 [21840/30000 (73%)]	Loss: 1.000999
Train Epoch: 75 [21920/30000 (73%)]	Loss: 0.000048
Train Epoch: 75 [22000/30000 (73%)]	Loss: 0.008509
Train Epoch: 75 [22080/30000 (74%)]	Loss: 0.006983
Train Epoch: 75 [22160/30000 (74%)]	Loss: 1.999987
Train Epoch: 75 [22240/30000 (74%)]	Loss: 1.010889
Train Epoch: 75 [22320/30000 (74%)]	Loss: 0.063174
Train Epoch: 75 [22400/30000 (75%)]	Loss: 1.000244
Train Epoch: 75 [22480/30000 (7

Train Epoch: 76 [4240/30000 (14%)]	Loss: 0.000403
Train Epoch: 76 [4320/30000 (14%)]	Loss: 0.999629
Train Epoch: 76 [4400/30000 (15%)]	Loss: 0.254005
Train Epoch: 76 [4480/30000 (15%)]	Loss: 1.080247
Train Epoch: 76 [4560/30000 (15%)]	Loss: 1.002721
Train Epoch: 76 [4640/30000 (15%)]	Loss: 1.000000
Train Epoch: 76 [4720/30000 (16%)]	Loss: 0.998736
Train Epoch: 76 [4800/30000 (16%)]	Loss: 1.006425
Train Epoch: 76 [4880/30000 (16%)]	Loss: 0.001006
Train Epoch: 76 [4960/30000 (17%)]	Loss: 0.062623
Train Epoch: 76 [5040/30000 (17%)]	Loss: 0.000172
Train Epoch: 76 [5120/30000 (17%)]	Loss: 0.001725
Train Epoch: 76 [5200/30000 (17%)]	Loss: 1.937180
Train Epoch: 76 [5280/30000 (18%)]	Loss: 0.000846
Train Epoch: 76 [5360/30000 (18%)]	Loss: 1.000356
Train Epoch: 76 [5440/30000 (18%)]	Loss: 2.000051
Train Epoch: 76 [5520/30000 (18%)]	Loss: 0.056191
Train Epoch: 76 [5600/30000 (19%)]	Loss: 3.013492
Train Epoch: 76 [5680/30000 (19%)]	Loss: 0.984125
Train Epoch: 76 [5760/30000 (19%)]	Loss: 0.960409


Train Epoch: 76 [17280/30000 (58%)]	Loss: 0.996276
Train Epoch: 76 [17360/30000 (58%)]	Loss: 0.032719
Train Epoch: 76 [17440/30000 (58%)]	Loss: 2.001281
Train Epoch: 76 [17520/30000 (58%)]	Loss: 1.000110
Train Epoch: 76 [17600/30000 (59%)]	Loss: 1.002547
Train Epoch: 76 [17680/30000 (59%)]	Loss: 1.005490
Train Epoch: 76 [17760/30000 (59%)]	Loss: 0.000003
Train Epoch: 76 [17840/30000 (59%)]	Loss: 0.000012
Train Epoch: 76 [17920/30000 (60%)]	Loss: 0.000054
Train Epoch: 76 [18000/30000 (60%)]	Loss: 0.999750
Train Epoch: 76 [18080/30000 (60%)]	Loss: 0.000598
Train Epoch: 76 [18160/30000 (61%)]	Loss: 0.004212
Train Epoch: 76 [18240/30000 (61%)]	Loss: 1.000594
Train Epoch: 76 [18320/30000 (61%)]	Loss: 0.000038
Train Epoch: 76 [18400/30000 (61%)]	Loss: 0.989239
Train Epoch: 76 [18480/30000 (62%)]	Loss: 0.003984
Train Epoch: 76 [18560/30000 (62%)]	Loss: 1.000161
Train Epoch: 76 [18640/30000 (62%)]	Loss: 0.000173
Train Epoch: 76 [18720/30000 (62%)]	Loss: 1.091607
Train Epoch: 76 [18800/30000 (6

Train Epoch: 77 [560/30000 (2%)]	Loss: 0.003484
Train Epoch: 77 [640/30000 (2%)]	Loss: 0.999999
Train Epoch: 77 [720/30000 (2%)]	Loss: 1.010852
Train Epoch: 77 [800/30000 (3%)]	Loss: 1.026171
Train Epoch: 77 [880/30000 (3%)]	Loss: 0.000022
Train Epoch: 77 [960/30000 (3%)]	Loss: 0.999999
Train Epoch: 77 [1040/30000 (3%)]	Loss: 0.007230
Train Epoch: 77 [1120/30000 (4%)]	Loss: 0.000296
Train Epoch: 77 [1200/30000 (4%)]	Loss: 0.069228
Train Epoch: 77 [1280/30000 (4%)]	Loss: 1.980898
Train Epoch: 77 [1360/30000 (5%)]	Loss: 0.003124
Train Epoch: 77 [1440/30000 (5%)]	Loss: 1.000089
Train Epoch: 77 [1520/30000 (5%)]	Loss: 0.138418
Train Epoch: 77 [1600/30000 (5%)]	Loss: 0.019536
Train Epoch: 77 [1680/30000 (6%)]	Loss: 0.272465
Train Epoch: 77 [1760/30000 (6%)]	Loss: 1.125661
Train Epoch: 77 [1840/30000 (6%)]	Loss: 1.004156
Train Epoch: 77 [1920/30000 (6%)]	Loss: 0.000004
Train Epoch: 77 [2000/30000 (7%)]	Loss: 2.000009
Train Epoch: 77 [2080/30000 (7%)]	Loss: 1.642880
Train Epoch: 77 [2160/3000

Train Epoch: 77 [14080/30000 (47%)]	Loss: 0.978173
Train Epoch: 77 [14160/30000 (47%)]	Loss: 0.000032
Train Epoch: 77 [14240/30000 (47%)]	Loss: 1.001292
Train Epoch: 77 [14320/30000 (48%)]	Loss: 1.000043
Train Epoch: 77 [14400/30000 (48%)]	Loss: 0.000053
Train Epoch: 77 [14480/30000 (48%)]	Loss: 0.061518
Train Epoch: 77 [14560/30000 (49%)]	Loss: 0.009557
Train Epoch: 77 [14640/30000 (49%)]	Loss: 1.000002
Train Epoch: 77 [14720/30000 (49%)]	Loss: 0.000000
Train Epoch: 77 [14800/30000 (49%)]	Loss: 1.000040
Train Epoch: 77 [14880/30000 (50%)]	Loss: 0.000554
Train Epoch: 77 [14960/30000 (50%)]	Loss: 1.000003
Train Epoch: 77 [15040/30000 (50%)]	Loss: 0.000000
Train Epoch: 77 [15120/30000 (50%)]	Loss: 0.001898
Train Epoch: 77 [15200/30000 (51%)]	Loss: 0.113743
